# Técnicas para codificar las variables categóricas (I): codificación ordinal y one-hot encoding

In [1]:
import pandas as pd

In [3]:
# Cargamos el dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning'
                 '-databases/adult/adult.data', encoding = 'utf-8',
                 header = None)


In [4]:
# Añadimos el nombre de cada variable
df.columns = ["age", "workclass", "fnlwgt", "education", 
              "education-num", "marital-status", "occupation", 
              "relationship", "race", "sex", "capital-gain", 
              "capital-loss", "hours-per-week", "native-country", 
              "class"]

In [5]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**Codificación ordinal**

Comenzamos con esta popular técnica de codificación que consiste simplemente en remplazar cada valor de la variable con un número entero distinto. Es por ello que la codificación ordinal resulta útil cuando tenemos datos ordinales, es decir, aquellos en los que se puede establecer un orden entre sus categóricas. En consecuencia, esta técnica nos ayudará a representar esta relación de orden de las distintas categorías.

In [6]:
from sklearn.preprocessing import OrdinalEncoder

# Creamos el codificador indicandole el orden de la variables
encoder = OrdinalEncoder(categories=[[" Preschool", " 1st-4th", 
                                      " 5th-6th", " 7th-8th", " 9th", 
                                      " 10th", " 11th", " 12th", 
                                      " HS-grad", " Some-college", 
                                      " Assoc-voc", " Assoc-acdm", 
                                      " Bachelors", " Masters", 
                                      " Prof-school", " Doctorate"]])

# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(df[["education"]])
df["education-encoded"] = encoder.transform(df[["education"]])

In [7]:
df[["education", "education-encoded"]].head(10)

,education,education-encoded
0,Bachelors,12.0
1,Bachelors,12.0
2,HS-grad,8.0
3,11th,6.0
4,Bachelors,12.0
5,Masters,13.0
6,9th,4.0
7,HS-grad,8.0
8,Masters,13.0
9,Bachelors,12.0


**Codification one-hot**

In [8]:
df.race.value_counts()

 White                 27816
 Black                  3124
 Asian-Pac-Islander     1039
 Amer-Indian-Eskimo      311
 Other                   271
Name: race, dtype: int64

In [9]:
# Creamos las variables binarias
dummies = pd.get_dummies(df['race'], drop_first = True)
dummies.head()

,Asian-Pac-Islander,Black,Other,White
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,1,0,0
4,0,1,0,0


In [10]:
# Añadimos las variables binarias al DataFrame
df = pd.concat([df, dummies], axis = 1)

# Eliminamos la vairable original race
df = df.drop(columns=['race'])

In [11]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,class,education-encoded,Asian-Pac-Islander,Black,Other,White
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K,12.0,0,0,0,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K,12.0,0,0,0,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K,8.0,0,0,0,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,United-States,<=50K,6.0,0,1,0,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Female,0,0,40,Cuba,<=50K,12.0,0,1,0,0


# Técnicas para codificar las variables categóricas (II): codificación binaria y hashing

Codificación binaria
Este tipo de codificación binaria es parecido a one-hot, pero en este caso las categorías se almacenan primero como códigos binarios. Los pasos que sigue este método para codificar son los siguientes:

Cada categoría única de nuestra variable categórica se convierte en un número ordinal sin tener en cuenta ningún orden. Así, este número irá desde el 1 hasta el número total de categorías únicas que tengamos.
Después, estos valores ordinales se transforman a su código binario. De esta manera, la categoría que le corresponda un valor de 10 se convertiría al código binario 1010.
Finalmente, cada dígito binario se separa en distintas columnas, una por cada dígito, por lo que para el código binario 1010 necesitaríamos 4 columnas.

In [13]:
from category_encoders import BinaryEncoder

# Creamos el codificador indicandole la columna
encoder = BinaryEncoder(cols=["native-country"])

# Ajustamos el codificador con native-country y la transformamos
encoder.fit(df["native-country"])
df_binario = encoder.transform(df["native-country"])
df = pd.concat([df, df_binario], axis=1)

In [14]:
df.loc[:, df.columns.str.startswith('native-country')].head()

,native-country,native-country_0,native-country_1,native-country_2,native-country_3,native-country_4,native-country_5
0,United-States,0,0,0,0,0,1
1,United-States,0,0,0,0,0,1
2,United-States,0,0,0,0,0,1
3,United-States,0,0,0,0,0,1
4,Cuba,0,0,0,0,1,0


**Codificación hashing**

In [15]:
from category_encoders import HashingEncoder

# Creamos el codificador indicandole el número de columnas
encoder = HashingEncoder(n_components=6)

# Ajustamos el codificador con native-country y la transformamos
encoder.fit(df["native-country"])
df_hash = encoder.transform(df["native-country"])
df = pd.concat([df, df_hash], axis=1)

In [16]:
df.loc[:, df.columns.str.startswith('col_') | 
       df.columns.str.startswith("native")].head(8)

,native-country,native-country_0,native-country_1,native-country_2,native-country_3,native-country_4,native-country_5,col_0,col_1,col_2,col_3,col_4,col_5
0,United-States,0,0,0,0,0,1,0,0,0,0,1,0
1,United-States,0,0,0,0,0,1,0,0,0,0,1,0
2,United-States,0,0,0,0,0,1,0,0,0,0,1,0
3,United-States,0,0,0,0,0,1,0,0,0,0,1,0
4,Cuba,0,0,0,0,1,0,0,1,0,0,0,0
5,United-States,0,0,0,0,0,1,0,0,0,0,1,0
6,Jamaica,0,0,0,0,1,1,0,0,1,0,0,0
7,United-States,0,0,0,0,0,1,0,0,0,0,1,0
